In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import pygal
import time
from collections import Counter
from matplotlib import pyplot as plt
from IPython.display import SVG, HTML, display

In [5]:
df = pd.read_csv('../data/train.csv')

In [6]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [5]:
# only 1/15 of the interactions have filters applied
df.count()

user_id            15932992
session_id         15932992
timestamp          15932992
step               15932992
action_type        15932992
reference          15932992
platform           15932992
city               15932992
device             15932992
current_filters     1153112
impressions         1586586
prices              1586586
dtype: int64

In [22]:
# split_timestamp = pd.Timestamp(year=2018, month=11, day=5, hour=16, minute=52, second=28)
split_timestamp = pd.Timestamp(year=2018, month=11, day=6)
split_time =time.mktime(split_timestamp.timetuple()) # Unix epoch time

print(split_timestamp)
print(split_time)

2018-11-06 00:00:00
1541455200.0


In [25]:
df_train = df[df['timestamp'] < split_time]
df_validation = df[df['timestamp'] >= split_time]

In [26]:
print('train set size: {} ({})'.format(len(df_train), len(df_train) / len(df)))
print('validation set size: {} ({})'.format(len(df_validation), len(df_validation) / len(df)))

train set size: 12838797 (0.8057995008093898)
validation set size: 3094195 (0.19420049919061028)


In [29]:
df_train.to_csv('../data/train_split.csv')

In [30]:
df_validation.to_csv('../data/validation_split.csv')

In [27]:
# for each session, if the last action type is "clickout item", delete the reference and save it for the labels
df_validation.set_index(keys=['session_id', 'step'])

user_id   timestamp              action_type  \
session_id    step                                                      
ad89aba9bb002 1     0BO8K084XKYE  1541457675          search for item   
              2     0BO8K084XKYE  1541458697   interaction item image   
              3     0BO8K084XKYE  1541458697   interaction item image   
              4     0BO8K084XKYE  1541458705            clickout item   
              5     0BO8K084XKYE  1541458805            clickout item   
74986cb514e04 1     0CI40GAHP96Q  1541456132   search for destination   
              2     0CI40GAHP96Q  1541456168            clickout item   
              3     0CI40GAHP96Q  1541456747   search for destination   
              4     0CI40GAHP96Q  1541456767   search for destination   
              5     0CI40GAHP96Q  1541456787     change of sort order   
              6     0CI40GAHP96Q  1541456826   search for destination   
              7     0CI40GAHP96Q  1541456884            clickout item   
0570c0ad1a064 1     0OQU0SD8LPT3  1541455534   interaction item image   
              2     0OQU0SD8LPT3  1541455534   interaction item image   
              3     0OQU0SD8LPT3  1541455534   interaction item image   
              4     0OQU0SD8LPT3  1541455534   interaction item image   
              5     0OQU0SD8LPT3  1541455534   interaction item image   
              6     0OQU0SD8LPT3  1541455534   interaction item image   
              7     0OQU0SD8LPT3  1541455534   interaction item image   
              8     0OQU0SD8LPT3  1541455544   interaction item image   
              9     0OQU0SD8LPT3  1541455544   interaction item image   
              10    0OQU0SD8LPT3  1541455544   interaction item image   
              11    0OQU0SD8LPT3  1541455544   interaction item image   
              12    0OQU0SD8LPT3  1541455544   interaction item image   
              13    0OQU0SD8LPT3  1541455544   interaction item image   
              14    0OQU0SD8LPT3  1541455544   interaction item image   
              15    0OQU0SD8LPT3  1541455544   interaction item image   
              16    0OQU0SD8LPT3  1541455544   interaction item image   
              17    0OQU0SD8LPT3  1541455544   interaction item image   
              18    0OQU0SD8LPT3  1541455544   interaction item image   
...                          ...         ...                      ...   
6b66cb0cfb518 7     ZU702MR210D2  1541498303   interaction item image   
              8     ZU702MR210D2  1541498303   interaction item image   
              9     ZU702MR210D2  1541498303  interaction item rating   
              10    ZU702MR210D2  1541498306            clickout item   
f701be9e58e9a 1     ZV6EJXAD9929  1541532352            clickout item   
e7916050980d9 1     ZVIDWWE0KWNB  1541531675   search for destination   
              2     ZVIDWWE0KWNB  1541531872          search for item   
              3     ZVIDWWE0KWNB  1541531973   interaction item deals   
              4     ZVIDWWE0KWNB  1541532069            clickout item   
061152606ba18 1     ZYDH48FW5Q3Q  1541545825   search for destination   
              2     ZYDH48FW5Q3Q  1541545935          search for item   
62728015bec05 1     ZYNMLE3MV3LK  1541544402   search for destination   
              2     ZYNMLE3MV3LK  1541544470   interaction item image   
              3     ZYNMLE3MV3LK  1541544470   interaction item image   
              4     ZYNMLE3MV3LK  1541544480   interaction item image   
              5     ZYNMLE3MV3LK  1541544480   interaction item image   
              6     ZYNMLE3MV3LK  1541544480   interaction item image   
              7     ZYNMLE3MV3LK  1541544480   interaction item image   
              8     ZYNMLE3MV3LK  1541544480   interaction item image   
              9     ZYNMLE3MV3LK  1541544480   interaction item image   
              10    ZYNMLE3MV3LK  1541544480   interaction item image   
              11    ZYNMLE3MV3LK  1541544480   interaction item image   
          